<a href="https://colab.research.google.com/github/ulfboge/malanje/blob/main/Extract_Area_From_Raster_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio geopandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 71.1 MB/s eta 0:00:00


In [ ]:
raster_path = '/content/fire_c.tif'
vector_path = '/content/crop_c.geojson'
output_path = '/content/your_raster_c.tif'


In [ ]:
import rasterio
from rasterio.features import geometry_mask
import geopandas as gpd
import numpy as np



In [ ]:
# Open the raster file
with rasterio.open(raster_path) as src:
    raster = src.read(1)  # Read the first band
    raster_meta = src.meta  # Save metadata for later use

    # Open the vector file
    vector = gpd.read_file(vector_path)

    # Ensure CRS matches
    vector = vector.to_crs(src.crs)

    # Create a mask from vector geometries
    mask_array = geometry_mask(
        [geom for geom in vector.geometry],  # List of geometries
        transform=src.transform,
        invert=False,  # Keep areas outside the polygons
        out_shape=(src.height, src.width)  # Match the raster dimensions
    )

    # Mask out areas inside polygons by setting them to nodata
    masked_raster = np.where(mask_array, raster, raster_meta['nodata'])

# Save the modified raster
raster_meta.update(dtype=raster.dtype)
with rasterio.open(output_path, 'w', **raster_meta) as dest:
    dest.write(masked_raster, 1)

print(f"Modified raster saved to {output_path}")

Modified raster saved to /content/your_raster_c.tif
